In [30]:
import numpy as np

In [55]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict):
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict)
    
    def __add__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, self, another)
        # if the other operand is not an Expression, then it must be a number
        # the number then should be converted to a Constant
        else:
            return Expression(Add, self, Constant(another))
    
    def __radd__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, another, self)
        else:
            return Expression(Add, Constant(another), self)
    
    def __sub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, self, another)
        else:
            return Expression(Sub, self, Constant(another))
    
    def __rsub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, another, self)
        else:
            return Expression(Sub, Constant(another), self)
        

#     def __mul__(self, other):
#         try:
#             return Expression(self.val * other.val, self.val * other.der + self.der * other.val)
#         except AttributeError:
#             return Expression(self.val * other, self.der * other)

#     def __rmul__(self, other):
#         return Expression(other * self.val, other * self.der)

In [56]:
class Variable(Expression):
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var, val_dict):
        return 1.0 if var is self else 0.0

In [68]:
class Constant(Expression):
    def __init__(self, val):
        self.val = val
        
    def evaluation_at(self, val_dict):
        return self.val
    
    def derivative_at(self, var, val_dict):
        return 0.0

In [69]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) + \
               sub_expr2.derivative_at(var, val_dict)

In [70]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) - \
               sub_expr2.derivative_at(var, val_dict)

In [71]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [72]:
def exp(expr):
    return Expression(Exp, expr)

Let $f(a, b) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [73]:
a = Variable()
b = Variable()
c = Variable()
f = exp(a-b+c)

In [74]:
f.evaluation_at({a: 1.0, b: 2.0, c: 3.0})

7.38905609893065

$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [75]:
f.derivative_at(b, {a: 1.0, b: 2.0, c: 3.0})

-7.38905609893065

$\dfrac{\partial f}{\partial a} = e^{a-b+c}$, at $(a, b) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [76]:
f.derivative_at(a, {a: 1.0, b: 2.0, c: 3.0})

7.38905609893065

Let $g(x, y) = x + e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $1+e \approx 3.718$.

In [79]:
x = Variable()
y = Variable()
g = x + exp(y-1)

In [80]:
g.evaluation_at({x: 1.0, y: 2.0})

3.718281828459045

$\dfrac{\partial g}{\partial x} = 1$

In [81]:
g.derivative_at(x, {x: 1.0, y: 2.0})

1.0

$\dfrac{\partial g}{\partial y} = e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $e^{1} \approx 2.718$.

In [83]:
g.derivative_at(y, {x: 1.0, y: 2.0})

2.718281828459045